In [30]:
using DataFrames, PyPlot, LightGraphs, Cairo, Fontconfig, Colors, CSV, RCall

In [9]:
include("code/BGSM_effective_resistance.jl");
include("code/model_selection.jl");

In [4]:
using PyCall, Suppressor
@pyimport GraphSegment
@pyimport numpy

In [7]:
edge = CSV.read("data/facebook_combined.txt", delim = ' ', header = false);

In [11]:
p = maximum(Matrix(edge)) + 1;
m = size(edge,1)
G = SimpleGraph(p);
for i = 1:m
    add_edge!(G,(edge[i,1] + 1, edge[i,2] + 1));
end
D = -incidence_matrix(G, oriented = true)';
R = effective_resistance(G, D);
m,p = size(D)
edges = repeat([(0,0)],m);
for i = 1:m
    temp = find(D[i,:] .!= 0);
    edges[i] = (temp[1]-1, temp[2]-1);
end

In [12]:
srand(4)
src = rand(1:p, 4); distance = zeros(p,4); col = zeros(Int, p);
for i = 1:4
    temp = dijkstra_shortest_paths(G, src[i]).dists;
    distance[:,i] = temp;
end
for i = 1:p
    col[i] = findmin(distance[i,:])[2]
end
col = [1;3;4;2][col];
tind = find(abs.(D * col) .> 1e-8)

651-element Array{Int64,1}:
   136
   422
   616
  1101
  1164
  1165
  1166
  1167
  1168
  1169
  1170
  1171
  1261
     ⋮
 37624
 37625
 37626
 37627
 37628
 37983
 37984
 37985
 37986
 37987
 37988
 37989

In [ ]:
srand(1);
y = 1 * col + rand(length(col));
bgsm = zeros(length(y),20);
for i = 1:20
    res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(i-1), verbose = false);
    mss1      = model_selection(res1);
    bgsm[:,i] = mss1[:beta]
    println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
end

In [ ]:
score: 12151.633900812532 and # of comp: 121
score: 12020.560921841163 and # of comp: 80
score: 11980.284708900073 and # of comp: 69
score: 11928.87261252071 and # of comp: 52
score: 11889.941024092788 and # of comp: 36
score: 11868.968094220054 and # of comp: 29
score: 11855.589736676691 and # of comp: 24
score: 11830.90384801837 and # of comp: 17
score: 11833.214245491812 and # of comp: 18
score: 11832.043330719634 and # of comp: 18
score: 11838.156592081814 and # of comp: 20
score: 11897.758839308966 and # of comp: 38
score: 11950.901300074196 and # of comp: 54
score: 11990.039511186842 and # of comp: 66
score: 12124.72318887452 and # of comp: 107
score: 12242.162047456217 and # of comp: 143
score: 12405.588282914472 and # of comp: 194
score: 12584.535457823442 and # of comp: 253
score: 12697.776412446008 and # of comp: 311
score: 12317.789141977904 and # of comp: 4039

In [14]:
srand(1);
y         = 1 * col +  rand(length(col));
res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(8-1), b = size(D,1), verbose = false);
mss1      = model_selection(res1);
println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
nzind     = find(abs.(D * mss1[:beta]) .> 1e-8);
tp        = length(findin(tind, nzind));
println("mse: ", norm(y - mss1[:beta])^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

score: 23033.07959219604 and # of comp: 3
mse: 0.21338918972677118 fdr: 0.0 power : 0.7357910906298003


In [13]:
srand(1)
y = 1 * col +  rand(length(col));
B     = 5;
err   = zeros(20, B);
out_l0pen = 0;
srand(1);
for b = 1:B
    println(b);
    for i = 1:20
        out_l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        sigma     = 1;
        alpha     = 0.2;
        z         = alpha * sigma * randn(p);
        y1        = y + z;
        y2        = y - z/alpha^2;
        mu1       = GraphSegment.GraphSegment(y1,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        err[i,b]  = norm(y2 - mu1)^2/p;
    end
end
l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

1
2
3
4
5


LoadError: [91msyntax: unexpected "end"[39m

In [15]:

l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

mse: 0.17913921989894008 fdr: 0.1235955056179775 power : 0.4792626728110599


In [ ]:
srand(1);
y = 2 * col + rand(length(col));
bgsm = zeros(length(y),20);
for i = 1:20
    res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-3 * 1.3^(i-1), verbose = false);
    mss1      = model_selection(res1);
    bgsm[:,i] = mss1[:beta]
    println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
end

In [ ]:
score: 12255.07631585365 and # of comp: 172
score: 12206.604339587655 and # of comp: 126
score: 12109.805660278449 and # of comp: 99
score: 12029.451524543623 and # of comp: 77
score: 11943.051013455246 and # of comp: 53
score: 11904.441390564012 and # of comp: 42
score: 11864.972103692558 and # of comp: 26
score: 11821.032574445853 and # of comp: 13
score: 11796.694905576913 and # of comp: 6
score: 11790.470361521864 and # of comp: 5
score: 11790.470361521864 and # of comp: 5
score: 11790.470361521864 and # of comp: 5
score: 11790.470361521864 and # of comp: 5
score: 11793.681987472992 and # of comp: 6
score: 11801.269862669367 and # of comp: 8
score: 11807.55635229327 and # of comp: 10
score: 11821.865050838645 and # of comp: 14
score: 11824.438092604189 and # of comp: 15
score: 11829.582439023 and # of comp: 17
score: 11843.857384477915 and # of comp: 21

In [17]:
srand(1);
y         = 2 * col +  rand(length(col));
res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(11-1), b = size(D,1), verbose = false);
mss1      = model_selection(res1);
println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
nzind     = find(abs.(D * mss1[:beta]) .> 1e-8);
tp        = length(findin(tind, nzind));
println("mse: ", norm(y - mss1[:beta])^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

score: 23027.79417807265 and # of comp: 4
mse: 0.08430369416781858 fdr: 0.006172839506172867 power : 0.989247311827957


In [18]:
srand(1)
y = 2 * col +  rand(length(col));
B     = 5;
err   = zeros(20, B);
out_l0pen = 0;
srand(1);
for b = 1:B
    println(b);
    for i = 1:20
        out_l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        sigma     = 1;
        alpha     = 0.2;
        z         = alpha * sigma * randn(p);
        y1        = y + z;
        y2        = y - z/alpha^2;
        mu1       = GraphSegment.GraphSegment(y1,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        err[i,b]  = norm(y2 - mu1)^2/p;
    end
end
l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

1
2
3
4
5
mse: 0.08538727086248098 fdr: 0.023400936037441533 power : 0.9615975422427036


In [ ]:
srand(1);
y = 3 * col + rand(length(col));
bgsm = zeros(length(y),20);
for i = 1:20
    res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-3 * 1.3^(i-1), verbose = false);
    mss1      = model_selection(res1);
    bgsm[:,i] = mss1[:beta]
    println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
end

In [ ]:
score: 11980.558854908842 and # of comp: 62
score: 11915.588746247817 and # of comp: 45
score: 11888.356506533179 and # of comp: 33
score: 11838.749165800391 and # of comp: 18
score: 11806.990479485921 and # of comp: 9
score: 11796.69490559552 and # of comp: 6
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11790.47036154062 and # of comp: 5
score: 11784.682700652913 and # of comp: 4
score: 11801.269870464053 and # of comp: 8
score: 11801.269870464053 and # of comp: 8
score: 11808.65111532856 and # of comp: 10
score: 11825.834698375442 and # of comp: 15
score: 11833.362919618381 and # of comp: 17
score: 11835.95579271776 and # of comp: 19

In [20]:
srand(1);
y         = 3 * col +  rand(length(col));
res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(13-1), b = size(D,1), verbose = false);
mss1      = model_selection(res1);
println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
nzind     = find(abs.(D * mss1[:beta]) .> 1e-8);
tp        = length(findin(tind, nzind));
println("mse: ", norm(y - mss1[:beta])^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

score: 23027.898359978382 and # of comp: 4
mse: 0.0854350377336627 fdr: 0.006172839506172867 power : 0.989247311827957


In [21]:
srand(1)
y = 3 * col +  rand(length(col));
B     = 5;
err   = zeros(20, B);
out_l0pen = 0;
srand(1);
for b = 1:B
    println(b);
    for i = 1:20
        out_l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        sigma     = 1;
        alpha     = 0.2;
        z         = alpha * sigma * randn(p);
        y1        = y + z;
        y2        = y - z/alpha^2;
        mu1       = GraphSegment.GraphSegment(y1,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        err[i,b]  = norm(y2 - mu1)^2/p;
    end
end
l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

1
2
3
4
5
mse: 0.08352323138298105 fdr: 0.0 power : 1.0


In [ ]:
srand(1);
y = 4 * col + rand(length(col));
bgsm = zeros(length(y),20);
for i = 1:20
    res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-3 * 1.3^(i-1), verbose = false);
    mss1      = model_selection(res1);
    bgsm[:,i] = mss1[:beta]
    println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
end

In [ ]:
score: 11861.347571671451 and # of comp: 25
score: 11821.656670445736 and # of comp: 13
score: 11796.69490562165 and # of comp: 6
score: 11790.470361566908 and # of comp: 5
score: 11790.470361566908 and # of comp: 5
score: 11784.682700678086 and # of comp: 4
score: 11784.682700678086 and # of comp: 4
score: 11784.682700678086 and # of comp: 4
score: 11790.470361566908 and # of comp: 5
score: 11790.470361566908 and # of comp: 5
score: 11790.470361566908 and # of comp: 5
score: 11790.470361566908 and # of comp: 5
score: 11790.470361566908 and # of comp: 5
score: 11784.682700678086 and # of comp: 4
score: 11795.168115065522 and # of comp: 6
score: 11801.269881970078 and # of comp: 8
score: 11812.000695589306 and # of comp: 11
score: 11829.370834485193 and # of comp: 16
score: 11836.95454120653 and # of comp: 18
score: 11836.048632331234 and # of comp: 19

In [22]:
srand(1);
y         = 4 * col +  rand(length(col));
res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(13-1), b = size(D,1), verbose = false);
mss1      = model_selection(res1);
println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
nzind     = find(abs.(D * mss1[:beta]) .> 1e-8);
tp        = length(findin(tind, nzind));
println("mse: ", norm(y - mss1[:beta])^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

score: 23028.04807279172 and # of comp: 4
mse: 0.0870609177212256 fdr: 0.006172839506172867 power : 0.989247311827957


In [23]:
srand(1)
y = 4 * col +  rand(length(col));
B     = 5;
err   = zeros(20, B);
out_l0pen = 0;
srand(1);
for b = 1:B
    println(b);
    for i = 1:20
        out_l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        sigma     = 1;
        alpha     = 0.2;
        z         = alpha * sigma * randn(p);
        y1        = y + z;
        y2        = y - z/alpha^2;
        mu1       = GraphSegment.GraphSegment(y1,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        err[i,b]  = norm(y2 - mu1)^2/p;
    end
end
l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

1
2
3
4
5
mse: 0.08352323138298105 fdr: 0.0 power : 1.0


In [ ]:
srand(1);
y = 5 * col + rand(length(col));
bgsm = zeros(length(y),20);
for i = 1:20
    res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-3 * 1.3^(i-1), verbose = false);
    mss1      = model_selection(res1);
    bgsm[:,i] = mss1[:beta]
    println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
end

In [24]:
srand(1);
y         = 5 * col +  rand(length(col));
res1      = BGSM_effective_resistance(y,D,R, v0 = 1e-2 * 1.3^(13-1), b = size(D,1), verbose = false);
mss1      = model_selection(res1);
println("score: ", mss1[:score]," and # of comp: ",length(mss1[:c]))
nzind     = find(abs.(D * mss1[:beta]) .> 1e-8);
tp        = length(findin(tind, nzind));
println("mse: ", norm(y - mss1[:beta])^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

score: 23028.243306386972 and # of comp: 4
mse: 0.08918133413050733 fdr: 0.006172839506172867 power : 0.989247311827957


In [25]:
srand(1)
y = 5 * col +  rand(length(col));
B     = 5;
err   = zeros(20, B);
out_l0pen = 0;
srand(1);
for b = 1:B
    println(b);
    for i = 1:20
        out_l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        sigma     = 1;
        alpha     = 0.2;
        z         = alpha * sigma * randn(p);
        y1        = y + z;
        y2        = y - z/alpha^2;
        mu1       = GraphSegment.GraphSegment(y1,edges, 1.5^(8-i), delta = 1, weights = R, verbose = false);
        err[i,b]  = norm(y2 - mu1)^2/p;
    end
end
l0pen = GraphSegment.GraphSegment(y,edges, 1.5^(8-findmin(sum(err,2))[2]), delta = 1, weights = R, verbose = false)
nzind = find(abs.(D * l0pen) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y-l0pen)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

1
2
3
4
5
mse: 0.08352323138298105 fdr: 0.0 power : 1.0


In [19]:
mse = zeros(5,3); fdr = zeros(5,3); power = zeros(5,3);

In [28]:
[mse fdr power]

5×9 Array{Float64,2}:
 0.213389   0.0  0.179139   0.0         …  0.123596   0.735791  0.0  0.479263
 0.0843037  0.0  0.0853873  0.00617284     0.0234009  0.989247  0.0  0.961598
 0.085435   0.0  0.0835232  0.00617284     0.0        0.989247  0.0  1.0     
 0.0870609  0.0  0.0835232  0.00617284     0.0        0.989247  0.0  1.0     
 0.0891813  0.0  0.0835232  0.00617284     0.0        0.989247  0.0  1.0     

In [26]:
mse[1,1] = 0.21338918972677118; fdr[1,1] = 0.0; power[1,1] = 0.7357910906298003;
mse[2,1] = 0.08430369416781858; fdr[2,1] = 0.006172839506172867; power[2,1] = 0.989247311827957;
mse[3,1] = 0.0854350377336627; fdr[3,1] = 0.006172839506172867; power[3,1] = 0.989247311827957;
mse[4,1] = 0.0870609177212256; fdr[4,1] = 0.006172839506172867; power[4,1] = 0.989247311827957;
mse[5,1] = 0.08918133413050733; fdr[5,1] = 0.006172839506172867; power[5,1] = 0.989247311827957;

In [27]:
mse[1,3] = 0.17913921989894008; fdr[1,3] = 0.1235955056179775; power[1,3] = 0.4792626728110599;
mse[2,3] = 0.08538727086248098; fdr[2,3] = 0.023400936037441533; power[2,3] = 0.9615975422427036;
mse[3,3] = 0.08352323138298105; power[3,3] = 1.0;
mse[4,3] = 0.08352323138298105; power[4,3] = 1.0
mse[5,3] = 0.08352323138298105; power[5,3] = 1.0;

In [32]:
X = pinv(full(D)); R"library(glmnet);"; @rput X;

4039×88234 Array{Float64,2}:
  0.00289791    0.000300549   0.00301747   …  -0.000129717   0.000422164
 -0.0644612     0.000233963  -0.000476691     -0.000129717   0.000422164
  0.00283132   -0.127743      0.00293125      -0.000129717   0.000422164
 -0.000596254   0.000214327  -0.064376        -0.000129717   0.000422164
  0.00265509    0.00029471    0.00275823      -0.000129717   0.000422164
 -0.0013572     0.000236377  -0.000485253  …  -0.000129717   0.000422164
  0.00288156   -0.00181777    0.0029969       -0.000129717   0.000422164
 -0.000565754   0.000214127  -0.000762637     -0.000129717   0.000422164
  0.00280505   -0.00021418    0.00289508      -0.000129717   0.000422164
 -0.00100621    0.000211917  -0.00310716      -0.000129717   0.000422164
 -0.000569644   0.000216368  -0.00176909   …  -0.000129717   0.000422164
  0.00289791    0.000300549   0.00301747      -0.000129717   0.000422164
  0.00289791    0.000300549   0.00301747      -0.000129717   0.000422164
  ⋮                   

In [35]:
y = 1 * col +  rand(length(col));
@rput y; 
R"y = y - mean(y);
res1 = cv.glmnet(X, y, family='gaussian', standardize = FALSE);";
R"genl = coef(res1)[-1];"; @rget genl;
nzind = find(abs.(genl) .> 1e-8);
println("mse: ", norm(y-X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 2 * col +  rand(length(col));
@rput y; 
R"y = y - mean(y);
res2 = cv.glmnet(X, y, family='gaussian', standardize = FALSE);";
R"genl = coef(res2)[-1];"; @rget genl;
nzind = find(abs.(genl) .> 1e-8);
println("mse: ", norm(y-X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 3 * col +  rand(length(col));
@rput y; 
R"y = y - mean(y);
res3 = cv.glmnet(X, y, family='gaussian', standardize = FALSE);";
R"genl = coef(res3)[-1];"; @rget genl;
nzind = find(abs.(genl) .> 1e-8);
println("mse: ", norm(y-X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 4 * col +  rand(length(col));
@rput y; 
R"y = y - mean(y);
res4 = cv.glmnet(X, y, family='gaussian', standardize = FALSE);";
R"genl = coef(res4)[-1];"; @rget genl;
nzind = find(abs.(genl) .> 1e-8);
println("mse: ", norm(y-X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 5 * col +  rand(length(col));
@rput y; 
R"y = y - mean(y);
res5 = cv.glmnet(X, y, family='gaussian', standardize = FALSE);";
R"genl = coef(res5)[-1];"; @rget genl;
nzind = find(abs.(genl) .> 1e-8);
println("mse: ", norm(y-X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

mse: 6.984472083790452 fdr: -35.166666666666664 power : 1.0
mse: 22.71608527725031 fdr: -39.6875 power : 1.0
mse: 47.479199695903745 fdr: -39.6875 power : 1.0
mse: 81.40120542125827 fdr: -42.4 power : 1.0
mse: 124.33981248739053 fdr: -39.6875 power : 1.0


In [39]:
nzind = find(abs.(genl) .> 1e-8);

In [ ]:
y = 1 * col +  rand(length(col));
R"genl = coef(res1)[-1];"; @rget genl;
nzind = find(abs.(D * X * genl) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y- mean(y) - X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 2 * col +  rand(length(col));
R"genl = coef(res2)[-1];"; @rget genl;
nzind = find(abs.(D * X * genl) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y- mean(y) - X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 3 * col +  rand(length(col));
R"genl = coef(res3)[-1];"; @rget genl;
nzind = find(abs.(D * X * genl) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y- mean(y) - X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 4 * col +  rand(length(col));
R"genl = coef(res4)[-1];"; @rget genl;
nzind = find(abs.(D * X * genl) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y- mean(y) - X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))
y = 5 * col +  rand(length(col));
R"genl = coef(res5)[-1];"; @rget genl;
nzind = find(abs.(D * X * genl) .> 1e-8);
tp    = length(findin(tind, nzind));
println("mse: ", norm(y- mean(y) - X * genl)^2/p," fdr: ", 1 - tp/length(nzind)," power : ", tp/length(tind))

In [52]:
df = DataFrame(mse = mse[:], fdr = fdr[:], power = power[:],
                signal = [1:5;1:5;1:5], method = repeat(["BGSM","Genlasso","L0pen"], inner = 5))
CSV.write("result/facebook1.txt", df, delim = ',');

In [57]:
sum(col .== 1), sum(col .== 2), sum(col .== 3), sum(col .== 4), 

(1758, 778, 750, 753)